In [13]:
pip install opencv-python


  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/ec/6c/fab8113424af5049f85717e8e527ca3773299a3c6b02506e66436e19874f/opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 487.6 kB/s eta 0:01:20
   ---------------------------------------- 0.1/38.8 MB 930.9 kB/s eta 0:00:42
   ---------------------------------------- 0.3/38.8 MB 1.8 MB/s eta 0:00:22
    --------------------------------------- 0.6/38.8 MB 2.7 MB/s eta 0:00:15
    --------------------------------------- 0.7/38.8 MB 2.6 MB/s eta 0:00:15
    --------------------------------------- 0.9/38.8 MB 2.9 MB/s eta 0:00:14
   - -------------------------------------- 1.1/38.8 MB 3.1 MB/s eta 0:00:13
   - ---------

In [10]:
import os
import cv2
import numpy as np


In [11]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [12]:
base_path = 'Pet_Images'
cat_path = os.path.join(base_path, 'cats')
dog_path = os.path.join(base_path, 'dogs')


In [14]:
def load_images_from_folder(folder):
    images = []
    labels = []
    if not os.path.exists(folder):
        print(f"Path {folder} does not exist.")
        return images, labels
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (128, 128))  # Resize image to 128x128
                img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
                hog_features = hog(img_gray, pixels_per_cell=(16, 16), cells_per_block=(2, 2), feature_vector=True)
                images.append(hog_features)
                if 'cats' in folder:
                    labels.append(0)  # Label 0 for cat
                elif 'dogs' in folder:
                    labels.append(1)  # Label 1 for dog
        except Exception as e:
            print(f"Error reading {img_path}: {e}")
    return images, labels


In [15]:
cat_images, cat_labels = load_images_from_folder(cat_path)
dog_images, dog_labels = load_images_from_folder(dog_path)


In [16]:
X = np.array(cat_images + dog_images)
y = np.array(cat_labels + dog_labels)


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [18]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='linear'))
])


In [19]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('scaler', StandardScaler()), ('svm', SVC(kernel='linear'))])

In [20]:
y_pred = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.637
              precision    recall  f1-score   support

           0       0.66      0.62      0.64       515
           1       0.62      0.65      0.64       485

    accuracy                           0.64      1000
   macro avg       0.64      0.64      0.64      1000
weighted avg       0.64      0.64      0.64      1000



In [21]:
param_grid = {'svm__C': [0.1, 1, 10, 100]}
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)


Best parameters: {'svm__C': 0.1}


In [22]:
y_pred = grid_search.predict(X_test)
print("Tuned Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Tuned Accuracy: 0.65
              precision    recall  f1-score   support

           0       0.67      0.64      0.65       515
           1       0.63      0.66      0.65       485

    accuracy                           0.65      1000
   macro avg       0.65      0.65      0.65      1000
weighted avg       0.65      0.65      0.65      1000

